# Integer Programming 
### The Beer Distribution Problem using PuLP
In this scenario we're looking to find which brewery warehouse should send shipments to each bars to minimize costs. There is a total of two warehouses(a and b) and a total of 5 bars. Each warehouse has a set amount of cases and each bar has a set amount to intake. 

In [1]:
from pulp import *

#### Setting supply nodes and number of units per

In [2]:
warehouses = ['a', 'b']
supply = {'a': 1000, 'b': 4000}
bars = ['1','2','3','4','5']

# Demand for each demand node
demand = {'1': 500, '2':900, '3':1800, '4':200, '5': 700}

# Costs of each transportation path
# Bars 1 2 3 4 5
# List 1 = warehouse a 
# List 2 = warehouse b
costs = [[2,4,5,2,1], [3,1,3,2,3]]

<img src="http://www.coin-or.org/PuLP/_images/brewery_nodes.jpg"/>

In [3]:
# Create a dictionary of costs
costs = makeDict([warehouses, bars], costs,0)

In [4]:
# Problem data, PuLP problem object creation for MIN
prob = LpProblem("Beer Distribution Problem", LpMinimize)
prob

Beer Distribution Problem:
MINIMIZE
None
VARIABLES

In [5]:
# Possible routes (transport)
routes = [(w,b) for w in warehouses for b in bars]
routes

[('a', '1'),
 ('a', '2'),
 ('a', '3'),
 ('a', '4'),
 ('a', '5'),
 ('b', '1'),
 ('b', '2'),
 ('b', '3'),
 ('b', '4'),
 ('b', '5')]

In [6]:
# Container for route variables 
vars_ = LpVariable.dicts("Route",(warehouses, bars),0,None,LpInteger)
vars_

{'a': {'1': Route_a_1,
  '2': Route_a_2,
  '3': Route_a_3,
  '4': Route_a_4,
  '5': Route_a_5},
 'b': {'1': Route_b_1,
  '2': Route_b_2,
  '3': Route_b_3,
  '4': Route_b_4,
  '5': Route_b_5}}

#### Objective function adding to prob 

In [7]:
prob += lpSum([vars_[w][b]*costs[w][b] for (w,b) in routes]), "Sum_of_Transportation_Costs"

#### Supply Maximum constraints for each warehouse

In [8]:
for w in warehouses:
    prob += lpSum([vars_[w][b] for b in bars]) <= supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w

#### Demand Minimum constraints for each bar

In [9]:
for b in bars:
    prob += lpSum([vars_[w][b] for w in warehouses]) >= demand[b], "Sum_of_Products_into_Bar%s"%b

In [10]:
# write to .lp file
prob.writeLP("BeerDistributionProblem.lp")

In [11]:
prob.solve()

1

In [12]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [13]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

Route_a_1 = 300.0
Route_a_2 = 0.0
Route_a_3 = 0.0
Route_a_4 = 0.0
Route_a_5 = 700.0
Route_b_1 = 200.0
Route_b_2 = 900.0
Route_b_3 = 1800.0
Route_b_4 = 200.0
Route_b_5 = 0.0


In [14]:
print("Total cost of transportation = ", value(prob.objective))

Total cost of transportation =  8600.0


## Resources
* https://github.com/ruxkor/pulp-or/blob/master/examples/BeerDistributionProblem.py
* http://www.coin-or.org/PuLP/CaseStudies/a_transportation_problem.html